# Tests Default Inventory Export

* Passes if unarchived inventory export matches the expected file contents

In [ ]:
#play
hosts: localhost
connection: local
gather_facts: no

In [ ]:
#task
set_fact:
  expected_inventory: "[all]\nlocalhost ansible_connection=local"
  exporter: ansible_kernel.exporters.ansible_zip.AnsibleZipExporter
  test_ipynb: |
    {
     "cells": [
      {
       "cell_type": "markdown",
       "metadata": {},
       "source": [
        "# Tests that a simple task runs\n",
        "\n",
        "* Passes if the debug command runs - PASS"
       ]
      },
      {
       "cell_type": "code",
       "execution_count": 1,
       "metadata": {},
       "outputs": [
        {
         "name": "stdout",
         "output_type": "stream",
         "text": [
          "TASK [debug] *******************************************************************\n",
          "ok: [localhost] => {\n",
          "    \"msg\": \"Hello world!\"\n",
          "}\n"
         ]
        }
       ],
       "source": [
        "debug:"
       ]
      },
      {
       "cell_type": "code",
       "execution_count": null,
       "metadata": {},
       "outputs": [],
       "source": []
      }
     ],
     "metadata": {
      "kernelspec": {
       "display_name": "Ansible",
       "language": "ansible",
       "name": "ansible"
      },
      "language_info": {
       "codemirror_mode": "yaml",
       "file_extension": ".yml",
       "mimetype": "text/yaml",
       "name": "ansible"
      }
     },
     "nbformat": 4,
     "nbformat_minor": 2
    }

In [ ]:
#task
shell: pwd
register: current_path_command

In [ ]:
#task
set_fact: current_path={{ current_path_command.stdout }}

In [ ]:
#task
name: create test file
copy: content="{{ test_ipynb }}" dest={{ current_path }}/test.ipynb

In [ ]:
#task
name: run exporter
shell: jupyter nbconvert --to {{ exporter }} {{ current_path }}/test.ipynb

In [ ]:
#task
name: unarchive test zip
unarchive:
  remote_src: yes
  src: '{{ current_path }}/test.zip'
  dest: '{{ current_path }}'

In [ ]:
#task
name: read inventory
slurp:
  src: '{{ current_path }}/inventory'
register: inventory_file

In [ ]:
#task
name: decode inventory
set_fact: actual_inventory={{ inventory_file['content'] | b64decode }}

In [ ]:
#task
name: check expected inventory content
assert:
  that: actual_inventory == expected_inventory